## 순환신경망 구현 및 학습

In [1]:
import tensorflow as tf

## 하이퍼 파라미터

In [2]:
EPOCHS = 10
NUM_WORDS = 10000

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)   # NUM_WORDS를 16개 사용한다.
    # Embedding : 양의 정수(색인)를 고정된 크기의 밀집 벡터로 전환.  # 길이가 10,000인 원핫 벡터를 길이가 16인 feature 벡터로 바꿔준다.
    # self.rnn = tf.keras.layers.SimpleRNN(32)
    # self.rnn = tf.keras.layers.LSTM(32)
    self.rnn = tf.keras.layers.GRU(32)
    # self.rnn1 = tf.keras.layers.LSTM(32)
    # self.rnn2 = tf.keras.layers.LSTM(32)
    # self.rnn3 = tf.keras.layers.LSTM(32)
    self.dense = tf.keras.layers.Dense(2, activation='softmax')

  def call(self, x, training=None, mask=None):
    x = self.emb(x)
    x = self.rnn(x)
    # x = self.rnn1(x)
    # x = self.rnn2(x)
    # x = self.rnn3(x)
    return self.dense(x)

## 학습, 테스트 루프 정의

In [4]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [5]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, value=0, padding='pre', maxlen=32)
# y가 다중 출력인 경우 padding='post' 로 하면 패딩이 뒤로 채워진다.
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, value=0, padding='pre', maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [6]:
# Create model
model = MyModel()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [7]:
for epoch in range(EPOCHS):
    for sequence, labels in train_ds:
        train_step(model, sequence, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_sequence, test_labels in test_ds:
        test_step(model, test_sequence, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.5098806023597717, Accuracy: 73.40399932861328, Test Loss: 0.4426090121269226, Test Accuracy: 79.15999603271484
Epoch 2, Loss: 0.3732762932777405, Accuracy: 83.2719955444336, Test Loss: 0.4466339349746704, Test Accuracy: 78.88400268554688
Epoch 3, Loss: 0.317598819732666, Accuracy: 86.36800384521484, Test Loss: 0.49060744047164917, Test Accuracy: 78.15599822998047
Epoch 4, Loss: 0.26839298009872437, Accuracy: 89.04000091552734, Test Loss: 0.5155260562896729, Test Accuracy: 77.3800048828125
Epoch 5, Loss: 0.22016486525535583, Accuracy: 91.36000061035156, Test Loss: 0.600638747215271, Test Accuracy: 76.57600402832031
Epoch 6, Loss: 0.17986884713172913, Accuracy: 93.23200225830078, Test Loss: 0.6372537612915039, Test Accuracy: 76.28399658203125
Epoch 7, Loss: 0.1415466070175171, Accuracy: 94.8479995727539, Test Loss: 0.7684795260429382, Test Accuracy: 75.43600463867188
Epoch 8, Loss: 0.10849899798631668, Accuracy: 96.22000122070312, Test Loss: 0.9271726608276367, Test Accu